In [3]:
import fitsio
import numpy as np
import healpy as hp
import pandas as pd
import matplotlib.pyplot as plt

#from base64 import b64encode
#import mechanize

%matplotlib inline

# I'm using Pandas (based on numpy), which is a very useful tool for
# handling tables and data.
# I'll properly put them into classes or more organized later

/Library/Python/2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#12GB file, be carefull
file_name = 'data/spAll-v5_10_0.fits'

fits = fitsio.FITS(file_name) 

# Columns we are interested on
cols = ['CLASS', 'RA','DEC','THING_ID','OBJTYPE','MJD','PLATE','FIBERID','BOSS_TARGET1','EBOSS_TARGET0','EBOSS_TARGET1']

In [3]:
# to show all names in file
#fits[1]

In [4]:
data  = fits[1].read(columns= cols)

In [5]:
%%time

d = {col:data[col].byteswap().newbyteorder() for col in cols}

CPU times: user 356 ms, sys: 375 ms, total: 731 ms
Wall time: 795 ms


In [6]:
#%%time

# Reading columns
#http://stackoverflow.com/questions/30283836/creating-pandas-dataframe-from-numpy-array-leads-to-strange-errors
#d = {col: fits[1][col].read().byteswap().newbyteorder() for col in cols}

In [7]:
df = pd.DataFrame(d)
#print 'number of rows =', len(df)
df.head()

,BOSS_TARGET1,CLASS,DEC,EBOSS_TARGET0,EBOSS_TARGET1,FIBERID,MJD,OBJTYPE,PLATE,RA,THING_ID
0,0,GALAXY,26.188042,0,2,1,57346,GALAXY,10000,30.012094,339167825
1,0,QSO,26.582945,0,1024,2,57346,QSO,10000,31.282286,342410358
2,0,GALAXY,26.608501,0,2,3,57346,GALAXY,10000,31.279594,342410449
3,0,QSO,26.682522,0,2048,4,57346,QSO,10000,31.237546,342675982
4,0,STAR,26.866054,0,2,5,57346,GALAXY,10000,31.394521,344112497


In [8]:
#%%time
#df.to_csv('subset_spAll-v5_10_0.csv', index=False)
#df = pd.read_csv('data/subset_spAll-v5_10_0.csv')

In [9]:
print ('number of rows =', len(df))
df.head()

('number of rows =', 3008000)


,BOSS_TARGET1,CLASS,DEC,EBOSS_TARGET0,EBOSS_TARGET1,FIBERID,MJD,OBJTYPE,PLATE,RA,THING_ID
0,0,GALAXY,26.188042,0,2,1,57346,GALAXY,10000,30.012094,339167825
1,0,QSO,26.582945,0,1024,2,57346,QSO,10000,31.282286,342410358
2,0,GALAXY,26.608501,0,2,3,57346,GALAXY,10000,31.279594,342410449
3,0,QSO,26.682522,0,2048,4,57346,QSO,10000,31.237546,342675982
4,0,STAR,26.866054,0,2,5,57346,GALAXY,10000,31.394521,344112497


## Computing pixels

In [10]:
'''map celestial coordinates to angular'''

#The resolution of the map
NSide = 2**5

phi_rad   = lambda ra : ra*np.pi/180.
theta_rad = lambda dec: (90.0 - dec)*np.pi/180.

df['PIX'] = hp.ang2pix(NSide, theta_rad(df['DEC']), phi_rad(df['RA']))

# Number of unique pixels
len(df['PIX'].unique())

3369

In [11]:
df = df.query('CLASS== "QSO".ljust(6) & (OBJTYPE=="QSO".ljust(16) | OBJTYPE=="NA".ljust(16)) & THING_ID != -1')
df.head()

,BOSS_TARGET1,CLASS,DEC,EBOSS_TARGET0,EBOSS_TARGET1,FIBERID,MJD,OBJTYPE,PLATE,RA,THING_ID,PIX
1,0,QSO,26.582945,0,1024,2,57346,QSO,10000,31.282286,342410358,3403
3,0,QSO,26.682522,0,2048,4,57346,QSO,10000,31.237546,342675982,3403
5,0,QSO,26.810259,0,1024,6,57346,QSO,10000,31.327011,344111998,3403
6,0,QSO,26.935936,0,2048,7,57346,QSO,10000,31.408747,345724597,3275
7,0,QSO,26.953807,0,1024,8,57346,QSO,10000,31.362048,345724589,3275


In [218]:
data = {'healp_1':[1,2,3], 'healp_2':[2,3,4]}
list(data)

['healp_1', 'healp_2']

In [ ]:
import fitsio
from fitsio import FITS,FITSHDR

In [463]:
%rm test3.fits
fits = FITS('test3.fits','rw')
nrows=1
data = dict()
#data = numpy.zeros(nrows, dtype=[('arr','S10',(1,4))])
data['arr'] = np.array(['abcd','1','2','3'],dtype=[('arr','S10',(1,4))]) #numpy.arange(nrows*1*4,dtype='f4').reshape(nrows,1,4)
#data['arr2'] = np.array('abca','2','3','4',dtype=[('arr2','S10',(1,4))])

# create a new table extension and write the data
fits.write(data)

TypeError: Unsupported numpy table datatype 20

In [451]:
data

{'arr': array([([['abcd', '', '', '']],), ([['1', '', '', '']],),
        ([['2', '', '', '']],), ([['3', '', '', '']],)], 
       dtype=[('arr', 'S10', (1, 4))]),
 'arr2': array([([['abca', '', '', '']],), ([['2', '', '', '']],),
        ([['3', '', '', '']],), ([['4', '', '', '']],)], 
       dtype=[('arr2', 'S10', (1, 4))])}

In [423]:
%rm test3.fits
data = {'healp_1':np.array([1,2,3]), 'healp_2':np.array([2,3,4])}
data['healp_4'] = np.array(['a','d','b'])
#data['healp_3'] = np.array([[[1,2,3],[2,3,4]]])
fdata   = data
fits = fitsio.FITS('test3.fits','rw')
fits.write(fdata)


In [445]:
fits = fitsio.FITS('test3.fits') 

In [446]:
fits[1]['arr'][:]

array([[['abcd      ', '1         ', '2         ', '3         ']]], 
      dtype='|S10')

In [464]:
d= {3403: {341840412: ['10000/spec-10000-57346-0100.fits', '5698/spec-5698-55947-0594.fits', '7691/spec-7691-57040-0093.fits', '7691/spec-7691-57360-0099.fits'], 340637645: ['10000/spec-10000-57346-0136.fits', '5697/spec-5697-55944-0969.fits', '7691/spec-7691-57040-0134.fits', '7691/spec-7691-57360-0132.fits'], 341840093: ['10000/spec-10000-57346-0092.fits', '5697/spec-5697-55944-0990.fits', '6270/spec-6270-56273-0202.fits', '7691/spec-7691-57040-0100.fits', '7691/spec-7691-57360-0094.fits']}}

In [468]:
d[3403][340637645]

['10000/spec-10000-57346-0136.fits',
 '5697/spec-5697-55944-0969.fits',
 '7691/spec-7691-57040-0134.fits',
 '7691/spec-7691-57360-0132.fits']

In [547]:
with open('master_0.csv', 'r') as myfile:
    data=myfile.read().replace('\n', '')

In [565]:
a = pd.read_json('{"0":{"0":[1,2,3],"1":3},"1":{"0":2,"1":4}}')

In [567]:
a[0][0]

[1, 2, 3]

,0
0,{3403: {341840412: ['10000/spec-10000-57346-01...
1,{3275: {349534382: ['10000/spec-10000-57346-09...
2,{3274: {350554689: ['10000/spec-10000-57346-06...
3,{3402: {341037487: ['10000/spec-10000-57346-04...


In [484]:
df = {3403: {341840412: ['10000/spec-10000-57346-0100.fits', '5698/spec-5698-55947-0594.fits', '7691/spec-7691-57040-0093.fits', '7691/spec-7691-57360-0099.fits'], 340637645: ['10000/spec-10000-57346-0136.fits', '5697/spec-5697-55944-0969.fits', '7691/spec-7691-57040-0134.fits', '7691/spec-7691-57360-0132.fits'], 341840093: ['10000/spec-10000-57346-0092.fits', '5697/spec-5697-55944-0990.fits', '6270/spec-6270-56273-0202.fits', '7691/spec-7691-57040-0100.fits', '7691/spec-7691-57360-0094.fits']}}
{3275: {349534382: ['10000/spec-10000-57346-0999.fits', '6270/spec-6270-56273-0074.fits', '6272/spec-6272-56325-0492.fits', '7691/spec-7691-57040-0996.fits', '7691/spec-7691-57360-0996.fits']}}
{3274: {350554689: ['10000/spec-10000-57346-0693.fits', '6270/spec-6270-56273-0438.fits', '7691/spec-7691-57040-0690.fits', '7691/spec-7691-57360-0702.fits'], 346239524: ['10000/spec-10000-57346-0436.fits', '5697/spec-5697-55944-0662.fits', '6268/spec-6268-56245-0140.fits', '7691/spec-7691-57040-0440.fits', '7691/spec-7691-57360-0438.fits'], 351278054: ['10000/spec-10000-57346-0604.fits', '6270/spec-6270-56273-0559.fits', '7691/spec-7691-57040-0607.fits', '7691/spec-7691-57360-0719.fits'], 350868168: ['10000/spec-10000-57346-0888.fits', '6270/spec-6270-56273-0238.fits', '7691/spec-7691-57040-0888.fits', '7691/spec-7691-57360-0932.fits'], 348420585: ['10000/spec-10000-57346-0536.fits', '6268/spec-6268-56245-0152.fits', '7691/spec-7691-57040-0536.fits', '7691/spec-7691-57360-0558.fits'], 351046414: ['10000/spec-10000-57346-0642.fits', '6270/spec-6270-56273-0600.fits', '7691/spec-7691-57040-0647.fits', '7691/spec-7691-57360-0711.fits'], 342652719: ['10000/spec-10000-57346-0172.fits', '5698/spec-5698-55947-0508.fits', '7691/spec-7691-57040-0180.fits', '7691/spec-7691-57360-0176.fits'], 347714116: ['10000/spec-10000-57346-0614.fits', '6270/spec-6270-56273-0491.fits', '7691/spec-7691-57040-0616.fits', '7691/spec-7691-57360-0636.fits'], 350283464: ['10000/spec-10000-57346-0732.fits', '6270/spec-6270-56273-0392.fits', '7691/spec-7691-57040-0730.fits', '7691/spec-7691-57360-0784.fits'], 345257490: ['10000/spec-10000-57346-0307.fits', '6270/spec-6270-56273-0378.fits', '7691/spec-7691-57040-0308.fits', '7691/spec-7691-57360-0308.fits'], 346955029: ['10000/spec-10000-57346-0280.fits', '6270/spec-6270-56273-0384.fits', '7691/spec-7691-57040-0280.fits', '7691/spec-7691-57360-0278.fits'], 341123990: ['10000/spec-10000-57346-0260.fits', '5697/spec-5697-55944-0914.fits', '7691/spec-7691-57040-0254.fits', '7691/spec-7691-57360-0260.fits'], 344738135: ['10000/spec-10000-57346-0266.fits', '6270/spec-6270-56273-0340.fits', '7691/spec-7691-57040-0262.fits', '7691/spec-7691-57360-0270.fits'], 353593796: ['10000/spec-10000-57346-0757.fits', '6270/spec-6270-56273-0676.fits', '7691/spec-7691-57040-0756.fits'], 350610703: ['10000/spec-10000-57346-0890.fits', '6270/spec-6270-56273-0240.fits', '7691/spec-7691-57040-0892.fits', '7691/spec-7691-57360-0926.fits'], 348052255: ['10000/spec-10000-57346-0818.fits', '6270/spec-6270-56273-0342.fits', '7691/spec-7691-57040-0818.fits', '7691/spec-7691-57360-0826.fits']}}
{3402: {341037487: ['10000/spec-10000-57346-0456.fits', '5697/spec-5697-55944-0672.fits', '7691/spec-7691-57040-0456.fits', '7691/spec-7691-57360-0448.fits'], 343269820: ['10000/spec-10000-57346-0482.fits', '6268/spec-6268-56245-0210.fits', '7691/spec-7691-57040-0486.fits', '7691/spec-7691-57360-0486.fits'], 336426277: ['10000/spec-10000-57346-0364.fits', '5697/spec-5697-55944-0814.fits', '7691/spec-7691-57040-0364.fits', '7691/spec-7691-57360-0364.fits'], 343388415: ['10000/spec-10000-57346-0484.fits', '6268/spec-6268-56245-0177.fits', '7691/spec-7691-57040-0490.fits', '7691/spec-7691-57360-0492.fits']}}

{3402: {336426277: ['10000/spec-10000-57346-0364.fits',
   '5697/spec-5697-55944-0814.fits',
   '7691/spec-7691-57040-0364.fits',
   '7691/spec-7691-57360-0364.fits'],
  341037487: ['10000/spec-10000-57346-0456.fits',
   '5697/spec-5697-55944-0672.fits',
   '7691/spec-7691-57040-0456.fits',
   '7691/spec-7691-57360-0448.fits'],
  343269820: ['10000/spec-10000-57346-0482.fits',
   '6268/spec-6268-56245-0210.fits',
   '7691/spec-7691-57040-0486.fits',
   '7691/spec-7691-57360-0486.fits'],
  343388415: ['10000/spec-10000-57346-0484.fits',
   '6268/spec-6268-56245-0177.fits',
   '7691/spec-7691-57040-0490.fits',
   '7691/spec-7691-57360-0492.fits']}}

In [486]:
df[3403]

{340637645: ['10000/spec-10000-57346-0136.fits',
  '5697/spec-5697-55944-0969.fits',
  '7691/spec-7691-57040-0134.fits',
  '7691/spec-7691-57360-0132.fits'],
 341840093: ['10000/spec-10000-57346-0092.fits',
  '5697/spec-5697-55944-0990.fits',
  '6270/spec-6270-56273-0202.fits',
  '7691/spec-7691-57040-0100.fits',
  '7691/spec-7691-57360-0094.fits'],
 341840412: ['10000/spec-10000-57346-0100.fits',
  '5698/spec-5698-55947-0594.fits',
  '7691/spec-7691-57040-0093.fits',
  '7691/spec-7691-57360-0099.fits']}

In [569]:
a = {3403: {341840412: ['10000/spec-10000-57346-0100.fits', '5698/spec-5698-55947-0594.fits', '7691/spec-7691-57040-0093.fits', '7691/spec-7691-57360-0099.fits'], 340637645: ['10000/spec-10000-57346-0136.fits', '5697/spec-5697-55944-0969.fits', '7691/spec-7691-57040-0134.fits', '7691/spec-7691-57360-0132.fits'], 341840093: ['10000/spec-10000-57346-0092.fits', '5697/spec-5697-55944-0990.fits', '6270/spec-6270-56273-0202.fits', '7691/spec-7691-57040-0100.fits', '7691/spec-7691-57360-0094.fits']}},{3275: {349534382: ['10000/spec-10000-57346-0999.fits', '6270/spec-6270-56273-0074.fits', '6272/spec-6272-56325-0492.fits', '7691/spec-7691-57040-0996.fits', '7691/spec-7691-57360-0996.fits']}},{3274: {350554689: ['10000/spec-10000-57346-0693.fits', '6270/spec-6270-56273-0438.fits', '7691/spec-7691-57040-0690.fits', '7691/spec-7691-57360-0702.fits'], 346239524: ['10000/spec-10000-57346-0436.fits', '5697/spec-5697-55944-0662.fits', '6268/spec-6268-56245-0140.fits', '7691/spec-7691-57040-0440.fits', '7691/spec-7691-57360-0438.fits'], 351278054: ['10000/spec-10000-57346-0604.fits', '6270/spec-6270-56273-0559.fits', '7691/spec-7691-57040-0607.fits', '7691/spec-7691-57360-0719.fits'], 350868168: ['10000/spec-10000-57346-0888.fits', '6270/spec-6270-56273-0238.fits', '7691/spec-7691-57040-0888.fits', '7691/spec-7691-57360-0932.fits'], 348420585: ['10000/spec-10000-57346-0536.fits', '6268/spec-6268-56245-0152.fits', '7691/spec-7691-57040-0536.fits', '7691/spec-7691-57360-0558.fits'], 351046414: ['10000/spec-10000-57346-0642.fits', '6270/spec-6270-56273-0600.fits', '7691/spec-7691-57040-0647.fits', '7691/spec-7691-57360-0711.fits'], 342652719: ['10000/spec-10000-57346-0172.fits', '5698/spec-5698-55947-0508.fits', '7691/spec-7691-57040-0180.fits', '7691/spec-7691-57360-0176.fits'], 347714116: ['10000/spec-10000-57346-0614.fits', '6270/spec-6270-56273-0491.fits', '7691/spec-7691-57040-0616.fits', '7691/spec-7691-57360-0636.fits'], 350283464: ['10000/spec-10000-57346-0732.fits', '6270/spec-6270-56273-0392.fits', '7691/spec-7691-57040-0730.fits', '7691/spec-7691-57360-0784.fits'], 345257490: ['10000/spec-10000-57346-0307.fits', '6270/spec-6270-56273-0378.fits', '7691/spec-7691-57040-0308.fits', '7691/spec-7691-57360-0308.fits'], 346955029: ['10000/spec-10000-57346-0280.fits', '6270/spec-6270-56273-0384.fits', '7691/spec-7691-57040-0280.fits', '7691/spec-7691-57360-0278.fits'], 341123990: ['10000/spec-10000-57346-0260.fits', '5697/spec-5697-55944-0914.fits', '7691/spec-7691-57040-0254.fits', '7691/spec-7691-57360-0260.fits'], 344738135: ['10000/spec-10000-57346-0266.fits', '6270/spec-6270-56273-0340.fits', '7691/spec-7691-57040-0262.fits', '7691/spec-7691-57360-0270.fits'], 353593796: ['10000/spec-10000-57346-0757.fits', '7691/spec-7691-57040-0756.fits', '6270/spec-6270-56273-0676.fits'], 350610703: ['10000/spec-10000-57346-0890.fits', '6270/spec-6270-56273-0240.fits', '7691/spec-7691-57040-0892.fits', '7691/spec-7691-57360-0926.fits'], 348052255: ['10000/spec-10000-57346-0818.fits', '6270/spec-6270-56273-0342.fits', '7691/spec-7691-57040-0818.fits', '7691/spec-7691-57360-0826.fits']}},{3402: {341037487: ['10000/spec-10000-57346-0456.fits', '5697/spec-5697-55944-0672.fits', '7691/spec-7691-57040-0456.fits', '7691/spec-7691-57360-0448.fits'], 343269820: ['10000/spec-10000-57346-0482.fits', '6268/spec-6268-56245-0210.fits', '7691/spec-7691-57040-0486.fits', '7691/spec-7691-57360-0486.fits'], 336426277: ['10000/spec-10000-57346-0364.fits', '5697/spec-5697-55944-0814.fits', '7691/spec-7691-57040-0364.fits', '7691/spec-7691-57360-0364.fits'], 343388415: ['10000/spec-10000-57346-0484.fits', '6268/spec-6268-56245-0177.fits', '7691/spec-7691-57040-0490.fits', '7691/spec-7691-57360-0492.fits']}}

In [9]:
b =pd.read_json('{"3403":{"th_341840093": ["10000/spec-10000-57346-0092.fits", "5697/spec-5697-55944-0990.fits", "6270/spec-6270-56273-0202.fits", "7691/spec-7691-57040-0100.fits", "7691/spec-7691-57360-0094.fits"], "th_341840412": ["10000/spec-10000-57346-0100.fits", "5698/spec-5698-55947-0594.fits", "7691/spec-7691-57040-0093.fits", "7691/spec-7691-57360-0099.fits"], "th_340637645": ["10000/spec-10000-57346-0136.fits", "5697/spec-5697-55944-0969.fits", "7691/spec-7691-57040-0134.fits", "7691/spec-7691-57360-0132.fits"]},"3275":{"th_349534382": ["10000/spec-10000-57346-0999.fits", "6270/spec-6270-56273-0074.fits", "6272/spec-6272-56325-0492.fits", "7691/spec-7691-57040-0996.fits", "7691/spec-7691-57360-0996.fits"]},"3274":{"th_353593796": ["10000/spec-10000-57346-0757.fits", "7691/spec-7691-57040-0756.fits", "6270/spec-6270-56273-0676.fits"], "th_350610703": ["10000/spec-10000-57346-0890.fits", "6270/spec-6270-56273-0240.fits", "7691/spec-7691-57040-0892.fits", "7691/spec-7691-57360-0926.fits"], "th_346955029": ["10000/spec-10000-57346-0280.fits", "6270/spec-6270-56273-0384.fits", "7691/spec-7691-57040-0280.fits", "7691/spec-7691-57360-0278.fits"], "th_347714116": ["10000/spec-10000-57346-0614.fits", "6270/spec-6270-56273-0491.fits", "7691/spec-7691-57040-0616.fits", "7691/spec-7691-57360-0636.fits"], "th_341123990": ["10000/spec-10000-57346-0260.fits", "5697/spec-5697-55944-0914.fits", "7691/spec-7691-57040-0254.fits", "7691/spec-7691-57360-0260.fits"], "th_350554689": ["10000/spec-10000-57346-0693.fits", "6270/spec-6270-56273-0438.fits", "7691/spec-7691-57040-0690.fits", "7691/spec-7691-57360-0702.fits"], "th_348420585": ["10000/spec-10000-57346-0536.fits", "6268/spec-6268-56245-0152.fits", "7691/spec-7691-57040-0536.fits", "7691/spec-7691-57360-0558.fits"], "th_351046414": ["10000/spec-10000-57346-0642.fits", "6270/spec-6270-56273-0600.fits", "7691/spec-7691-57040-0647.fits", "7691/spec-7691-57360-0711.fits"], "th_351278054": ["10000/spec-10000-57346-0604.fits", "6270/spec-6270-56273-0559.fits", "7691/spec-7691-57040-0607.fits", "7691/spec-7691-57360-0719.fits"], "th_342652719": ["10000/spec-10000-57346-0172.fits", "5698/spec-5698-55947-0508.fits", "7691/spec-7691-57040-0180.fits", "7691/spec-7691-57360-0176.fits"], "th_350868168": ["10000/spec-10000-57346-0888.fits", "6270/spec-6270-56273-0238.fits", "7691/spec-7691-57040-0888.fits", "7691/spec-7691-57360-0932.fits"], "th_345257490": ["10000/spec-10000-57346-0307.fits", "6270/spec-6270-56273-0378.fits", "7691/spec-7691-57040-0308.fits", "7691/spec-7691-57360-0308.fits"], "th_350283464": ["10000/spec-10000-57346-0732.fits", "6270/spec-6270-56273-0392.fits", "7691/spec-7691-57040-0730.fits", "7691/spec-7691-57360-0784.fits"], "th_348052255": ["10000/spec-10000-57346-0818.fits", "6270/spec-6270-56273-0342.fits", "7691/spec-7691-57040-0818.fits", "7691/spec-7691-57360-0826.fits"], "th_344738135": ["10000/spec-10000-57346-0266.fits", "6270/spec-6270-56273-0340.fits", "7691/spec-7691-57040-0262.fits", "7691/spec-7691-57360-0270.fits"], "th_346239524": ["10000/spec-10000-57346-0436.fits", "5697/spec-5697-55944-0662.fits", "6268/spec-6268-56245-0140.fits", "7691/spec-7691-57040-0440.fits", "7691/spec-7691-57360-0438.fits"]}}')

In [66]:
file = open('master_0.csv', 'r')
healpix=[]
for line in file:
    healpix.append(pd.read_json(line))

In [70]:
healpix[0]

,3403
th_340637645,"[10000/spec-10000-57346-0136.fits, 5697/spec-5..."
th_341840093,"[10000/spec-10000-57346-0092.fits, 5697/spec-5..."
th_341840412,"[10000/spec-10000-57346-0100.fits, 5698/spec-5..."


In [69]:
healpix[1]

,3275
th_349534382,"[10000/spec-10000-57346-0999.fits, 6270/spec-6..."
